### Installation

In [ ]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth==2025.3.14
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.3.13 requires tyro, which is not installed.
unsloth-zoo 2025.3.13 requires protobuf<4.0.0

### Unsloth


In [ ]:
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.14: Fast Gemma3 patching. Transformers: 4.50.0.dev0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/192 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

Add LoRA adapters

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!
    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


<a name="Data"></a>
### Data Prep
 `Gemma-3` format

```
<bos><start_of_turn>user
Hello!<end_of_turn>
<start_of_turn>model
Hey there!<end_of_turn>
```

use `get_chat_template` function to get the correct chat template.

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [ ]:
from datasets import load_dataset
dataset = load_dataset("hungsvdut2k2/vietnamese-medical-chat-data", split = "train")


README.md:   0%|          | 0.00/355 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/28.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/46479 [00:00<?, ? examples/s]

use `standardize_data_formats` to try converting datasets to the correct format for finetuning purposes!

In [ ]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

In [ ]:
dataset[100]

In [ ]:
def apply_chat_template(examples):
    texts = tokenizer.apply_chat_template(examples["conversation"])
    return { "text" : texts }
pass
dataset = dataset.map(apply_chat_template, batched = True)

Map:   0%|          | 0/46479 [00:00<?, ? examples/s]

In [ ]:
dataset[100]["text"]

<a name="Train"></a>
### Train the model
use Huggingface TRL's `SFTTrainer`

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16
Unsloth: We found double BOS tokens - we shall remove one automatically.


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/46479 [00:00<?, ? examples/s]

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=2):   0%|          | 0/46479 [00:00<?, ? examples/s]

In [ ]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<bos><start_of_turn>user\nBạn có thể cho tôi biết tên biệt dược của Nicardipine được không?<end_of_turn>\n<start_of_turn>model\nChắc chắn rồi! Một số tên thương hiệu phổ biến của Nicardipine bao gồm Cardene và Carden SR. Bạn còn muốn biết điều gì khác về Nicardipine không?<end_of_turn>\n<start_of_turn>user\nBạn có thể cho tôi biết về công dụng của Nicardipine được không?<end_of_turn>\n<start_of_turn>model\nNicardipine thường được sử dụng để điều trị huyết áp cao và một số loại đau thắt ngực, là một loại đau ngực xảy ra khi lưu lượng máu đến tim giảm. Nó hoạt động bằng cách thư giãn mạch máu và tăng lưu lượng máu đến tim.<end_of_turn>\n<start_of_turn>user\nSử dụng Nicardipine có tác dụng phụ nào không?<end_of_turn>\n<start_of_turn>model\nCó, có một số tác dụng phụ tiềm ẩn liên quan đến việc sử dụng Nicardipine. Chúng có thể bao gồm nhức đầu, chóng mặt, đỏ bừng, buồn nôn và đau dạ dày, cùng nhiều triệu chứng khác. Điều quan trọng là phải nói chuyện với bác sĩ về bất kỳ tác dụng phụ tiềm

In [ ]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[50]["labels"]]).replace(tokenizer.pad_token, " ")

'                                Cabergoline thường được sử dụng để điều trị nồng độ prolactin cao. Liều lượng có thể khác nhau tùy thuộc vào từng cá nhân và mức độ nghiêm trọng của tình trạng của họ. Thông thường, liều ban đầu là 0,5 miligam mỗi tuần, có thể tăng lên cho đến khi đạt được hiệu quả mong muốn.<end_of_turn>\n                     Có, Cabergoline có những tác dụng phụ tiềm ẩn mà bạn nên biết. Một số tác dụng phụ thường gặp bao gồm buồn nôn, chóng mặt và đau đầu. Trong một số ít trường hợp, các tác dụng phụ nghiêm trọng hơn như các vấn đề về van tim và ảo giác đã được báo cáo.<end_of_turn>\n                   Thông thường, Cabergoline được dùng một hoặc hai lần một tuần, theo chỉ định của chuyên gia chăm sóc sức khỏe. Có thể uống cùng hoặc không cùng thức ăn, nhưng nên uống vào cùng một thời điểm mỗi tuần để duy trì hàm lượng ổn định trong cơ thể.<end_of_turn>\n                   Không, phụ nữ mang thai không nên dùng Cabergoline vì nó có thể gây hại cho thai nhi. Nếu bạn đa

Train the model

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 46,479 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 14,901,248/4,000,000,000 (0.37% trained)
It is strongly recommended to train Gemma3 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,9.110000
2,9.328500
3,8.553500
4,8.931800
5,8.209700
6,9.154600
7,9.154700
8,8.551300
9,9.213300
10,9.067600


<a name="Save"></a>
### Saving, loading gemma-3-lora

In [ ]:
!pip install huggingface_hub


In [ ]:
from huggingface_hub import login
login()
#get token : https://huggingface.co/settings/tokens

In [ ]:
model.push_to_hub("gemma-3-lora") # Online saving
tokenizer.push_to_hub("gemma-3-lora") # Online saving

Saved model to https://huggingface.co/gemma-3-lora


In [ ]:
from peft import PeftModel
from unsloth import FastModel

# FastModel.from_pretrained now returns a single model object
base_model, tokenizer  = FastModel.from_pretrained("unsloth/gemma-3-4b-it", load_in_4bit=True)

model = PeftModel.from_pretrained(base_model, "NAME YOUR MODEL IN HUGGINGFACE")

==((====))==  Unsloth 2025.3.14: Fast Gemma3 patching. Transformers: 4.50.0.dev0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


adapter_config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/59.7M [00:00<?, ?B/s]

In [ ]:
from transformers import TextStreamer

messages = [{
    "role": "user",
    "content": [{"type": "text", "text": "Tôi nghĩ tôi đã bị sốt xuất huyết"}]
}]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True
)

inputs = tokenizer([text], return_tensors="pt").to("cuda")

streamer = TextStreamer(tokenizer, skip_prompt=True)
outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    temperature=1.0,
    top_p=0.95,
    top_k=64,
    streamer=streamer
)


Tôi hiểu bạn đang lo lắng về việc có thể bị sốt xuất huyết. Đây là một tình trạng nghiêm trọng và cần được xử lý đúng cách. Dưới đây là những thông tin và lời khuyên tôi có thể cung cấp cho bạn:

**1. Triệu chứng của sốt xuất huyết:**

Sốt xuất huyết có thể có nhiều dạng, nhưng các triệu chứng phổ biến bao gồm:

*   **Sốt:** Thường cao, đột ngột và kéo dài.
*   **Đau đầu:** Thường dữ dội.
*   **Đau cơ:** Đau nhức toàn thân, đặc biệt là ở các cơ.
*   **Chills (runniness):** Có cảm giác lạnh toát, run rẩy.
*   **Buồn nôn, nôn:** Có thể xảy ra.
*   **Tiêu chảy:** Cũng có thể xảy ra.
*   **Mệt mỏi:**
